# 数值分析 实验4
2019011265 计93 丁韶峰

In [11]:
import numpy as np

In [12]:
n = 100
a = 0.5
h = 1 / n

In [25]:
def gen_data(eps):
  A = np.zeros((n - 1, n - 1))
  for i in range(n - 1):
    if i != 0:
      A[i][i - 1] = eps
    A[i][i] = -(2 * eps + h)
    if i != n - 2:
      A[i][i + 1] = eps + h
  b = np.zeros(n - 1)
  for i in range(n - 1):
    b[i] = a * h * h
    if i == n - 2:
      b[i] -= eps + h
  return A, b

In [14]:
def accurate_sol(x, eps):
  return (1 - a) / (1 - np.exp(-1 / eps)) * (1 - np.exp(-x / eps)) + a * x

In [15]:
def jacobi(A, b, n):
  x = np.ones_like(b)
  cnt = 0
  while True:
    y = np.copy(x)
    for i in range(n):
      x[i] = b[i]
      if i != 0:
        x[i] -= A[i][i - 1] * y [i - 1]
      if i != n - 1:
        x[i] -= A[i][i + 1] * y[i + 1]
      x[i] /= A[i][i]
    cnt += 1
    if np.array_equiv(np.around(x, decimals=4), np.around(y, decimals=4)):
      print("Jacobi stops after {} iterations".format(cnt))
      return x

In [16]:
def gs(A, b, n):
  x = np.ones_like(b)
  cnt = 0
  while True:
    y = np.copy(x)
    for i in range(n):
      x[i] = b[i]
      if i != 0:
        x[i] -= A[i][i - 1] * x[i - 1]
      if i != n - 1:
        x[i] -= A[i][i + 1] * x[i + 1]
      x[i] /= A[i][i]
    cnt += 1
    if np.array_equiv(np.around(x, decimals=4), np.around(y, decimals=4)):
      print("GS stops after {} iterations".format(cnt))
      return x

In [17]:
def sor(A, b, omega, n):
  x = np.ones_like(b)
  cnt = 0
  while True:
    y = np.copy(x)
    for i in range(n):
      x[i] = b[i]
      if i != 0:
        x[i] -= A[i][i - 1] * x[i - 1]
      if i != n - 1:
        x[i] -= A[i][i + 1] * x[i + 1]
      x[i] /= A[i][i]
      x[i] = (1 - omega) * y[i] + omega * x[i]
    cnt += 1
    if np.array_equiv(np.around(x, decimals=4), np.around(y, decimals=4)):
      print("SOR stops after {} iterations".format(cnt))
      return x

In [36]:
def compute_arr(appro_sol, acc_sol):
  appro_sol = appro_sol.reshape(np.shape(acc_sol))
  inv_norm = np.max(np.abs(appro_sol - acc_sol))
  two_norm = np.linalg.norm(appro_sol - acc_sol)
  return inv_norm, two_norm

In [34]:
def compute(eps):
  A, b = gen_data(eps)
  acc_sol = [accurate_sol(x, eps) for x in np.arange(h, 1, h)]
  jacobi_sol = jacobi(A, b, n - 1)
  gs_sol = gs(A, b, n - 1)
  sor_sol = sor(A, b, 1.1, n - 1)
  jacobi_inv, jacobi_two = compute_arr(jacobi_sol, acc_sol)
  gs_inv, gs_two = compute_arr(gs_sol, acc_sol)
  sor_inv, sor_two = compute_arr(sor_sol, acc_sol)
  print("jacobi error: 2 norm {}, inv norm {}".format(jacobi_inv, jacobi_two))
  print("gs error: 2 norm {}, inv norm {}".format(gs_inv, gs_two))
  print("sor error: 2 norm {}, inv norm {}".format(sor_inv, sor_two))

In [41]:
compute(1)

Jacobi stops after 6771 iterations
GS stops after 3937 iterations
SOR stops after 3278 iterations
jacobi error: 2 norm 0.017719708807683032, inv norm 0.12471265293007759
gs error: 2 norm 0.009821996442479053, inv norm 0.06902704596848404
sor error: 2 norm 0.009091856070018278, inv norm 0.06387801248061144
